In [1]:
from numpy import ascontiguousarray as ascont
import numpy as np
from sigmaepsilon.mesh.grid import grid
from sigmaepsilon.mesh import triangulate
from sigmaepsilon.mesh.utils.topology.tr import Q4_to_T3

from sigmaepsilon.solid.material import MindlinPlateSection as Section
from sigmaepsilon.math.linalg import ReferenceFrame
from sigmaepsilon.solid.material import (
    ElasticityTensor,
    LinearElasticMaterial,
    HuberMisesHenckyFailureCriterion_SP,
)
from sigmaepsilon.solid.material.utils import elastic_stiffness_matrix

from sigmaepsilon.solid.fourier import (
    RectangularPlate,
    LoadGroup,
    PointLoad,
    RectangleLoad,
)

size = Lx, Ly = (600.0, 800.0)
E = 2890.0
nu = 0.2
t = 25.0
yield_strength = 25.0
grid_shape = (50, 50)
num_group = 200
max_num_cases_per_group = 10

hooke = elastic_stiffness_matrix(E=E, NU=nu)
frame = ReferenceFrame(dim=3)
stiffness = ElasticityTensor(hooke, frame=frame, tensorial=False)

failure_model = HuberMisesHenckyFailureCriterion_SP(yield_strength=yield_strength)

material = LinearElasticMaterial(stiffness=stiffness, failure_model=failure_model)

section = Section(
    layers=[
        Section.Layer(material=material, thickness=t),
    ]
)
ABDS = section.elastic_stiffness_matrix()
D, S = ascont(ABDS[:3, :3]), ascont(ABDS[3:, 3:])


f:\GitHub\SigmaEpsilon\sigmaepsilon.solid.fourier\.solid.fourier\lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [2]:
load_groups = dict()
case_counter, group_counter = 0, 0
for i in range(num_group):
    group_counter += 1
    load_cases = dict()
    number_of_cases = np.random.choice(np.arange(max_num_cases_per_group)+1)
    for j in range(number_of_cases):
        case_counter += 1
        if np.random.rand() > 0.5:
            xA, yA = np.random.rand() * Lx / 2, np.random.rand() * Ly / 2
            xB, yB = xA + np.random.rand() * Lx / 2, yA + np.random.rand() * Ly / 2
            v = np.random.rand(3)
            load_case = RectangleLoad(x=[[xA, yA], [xB, yB]], v=v)
        else:
            load_case = PointLoad(x=[Lx / 3, Ly / 2], v=[-100.0, 0, 0])
        load_cases[f"Case {case_counter}"] = load_case
    load_groups[f"Group {group_counter}"] = LoadGroup(**load_cases)

loads = LoadGroup(**load_groups)
loads.lock()

print(f"Added {case_counter} cases in {group_counter} groups.")

Added 1090 cases in 200 groups.


In [3]:
from sigmaepsilon.deepdict import asciiprint
asciiprint(loads)

LoadGroup
 +-- Group 1
 |   +-- Case 1
 |   +-- Case 2
 |   +-- Case 3
 |   +-- Case 4
 |   +-- Case 5
 |   +-- Case 6
 |   +-- Case 7
 |   +-- Case 8
 |   +-- Case 9
 +-- Group 2
 |   +-- Case 10
 |   +-- Case 11
 +-- Group 3
 |   +-- Case 12
 |   +-- Case 13
 |   +-- Case 14
 +-- Group 4
 |   +-- Case 15
 |   +-- Case 16
 |   +-- Case 17
 |   +-- Case 18
 |   +-- Case 19
 +-- Group 5
 |   +-- Case 20
 |   +-- Case 21
 |   +-- Case 22
 |   +-- Case 23
 |   +-- Case 24
 |   +-- Case 25
 |   +-- Case 26
 |   +-- Case 27
 |   +-- Case 28
 +-- Group 6
 |   +-- Case 29
 |   +-- Case 30
 |   +-- Case 31
 |   +-- Case 32
 +-- Group 7
 |   +-- Case 33
 |   +-- Case 34
 |   +-- Case 35
 |   +-- Case 36
 |   +-- Case 37
 |   +-- Case 38
 |   +-- Case 39
 +-- Group 8
 |   +-- Case 40
 |   +-- Case 41
 |   +-- Case 42
 |   +-- Case 43
 |   +-- Case 44
 |   +-- Case 45
 |   +-- Case 46
 |   +-- Case 47
 |   +-- Case 48
 +-- Group 9
 |   +-- Case 49
 |   +-- Case 50
 |   +-- Case 51
 +-- Group 10
 

In [4]:
coords, triangles = Q4_to_T3(*grid(size=size, shape=grid_shape, eshape="Q4"))
triobj = triangulate(points=coords[:, :2], triangles=triangles)[-1]
plate = RectangularPlate(size, (20, 20), D=D, S=S)
loads.problem = plate

In [5]:
# the first load group
next(loads.values())

LoadGroup({'Case 1': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 2': RectangleLoad({'x': [[104.58501059326417, 334.7006105962845], [165.25883822635677, 398.29663401029376]], 'v': array([0.53187527, 0.12620011, 0.48822345])}), 'Case 3': RectangleLoad({'x': [[48.959625850519174, 67.91078162868875], [300.6012248986844, 451.76066889241594]], 'v': array([0.30388577, 0.09970683, 0.52461826])}), 'Case 4': RectangleLoad({'x': [[196.90748880275964, 320.80144724613797], [304.3377719429589, 534.1659395352261]], 'v': array([0.3604886 , 0.59257619, 0.16958371])}), 'Case 5': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 6': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 7': RectangleLoad({'x': [[54.030148129440626, 68.81200393827203], [97.24207408564268, 92.01555185605565]], 'v': array([0.42785045, 0.05710801, 0.07006977])}), 'Case 8': RectangleLoad({'x': [[284.3334763991404, 23.082442148770088], [303.24173210567227, 55.03048055075901]], 'v': array([0.43

In [6]:
[b for b in loads.blocks()]

[LoadGroup({'Case 1': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 2': RectangleLoad({'x': [[104.58501059326417, 334.7006105962845], [165.25883822635677, 398.29663401029376]], 'v': array([0.53187527, 0.12620011, 0.48822345])}), 'Case 3': RectangleLoad({'x': [[48.959625850519174, 67.91078162868875], [300.6012248986844, 451.76066889241594]], 'v': array([0.30388577, 0.09970683, 0.52461826])}), 'Case 4': RectangleLoad({'x': [[196.90748880275964, 320.80144724613797], [304.3377719429589, 534.1659395352261]], 'v': array([0.3604886 , 0.59257619, 0.16958371])}), 'Case 5': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 6': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 7': RectangleLoad({'x': [[54.030148129440626, 68.81200393827203], [97.24207408564268, 92.01555185605565]], 'v': array([0.42785045, 0.05710801, 0.07006977])}), 'Case 8': RectangleLoad({'x': [[284.3334763991404, 23.082442148770088], [303.24173210567227, 55.03048055075901]], 'v': array([0.4

In [7]:
[b for b in loads.blocks(inclusive=True, deep=True)]

[LoadGroup({'Group 1': LoadGroup({'Case 1': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 2': RectangleLoad({'x': [[104.58501059326417, 334.7006105962845], [165.25883822635677, 398.29663401029376]], 'v': array([0.53187527, 0.12620011, 0.48822345])}), 'Case 3': RectangleLoad({'x': [[48.959625850519174, 67.91078162868875], [300.6012248986844, 451.76066889241594]], 'v': array([0.30388577, 0.09970683, 0.52461826])}), 'Case 4': RectangleLoad({'x': [[196.90748880275964, 320.80144724613797], [304.3377719429589, 534.1659395352261]], 'v': array([0.3604886 , 0.59257619, 0.16958371])}), 'Case 5': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 6': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 7': RectangleLoad({'x': [[54.030148129440626, 68.81200393827203], [97.24207408564268, 92.01555185605565]], 'v': array([0.42785045, 0.05710801, 0.07006977])}), 'Case 8': RectangleLoad({'x': [[284.3334763991404, 23.082442148770088], [303.24173210567227, 55.03048055075

In [8]:
[b for b in loads.cases(inclusive=True, deep=True)]

[PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}),
 RectangleLoad({'x': [[104.58501059326417, 334.7006105962845], [165.25883822635677, 398.29663401029376]], 'v': array([0.53187527, 0.12620011, 0.48822345])}),
 RectangleLoad({'x': [[48.959625850519174, 67.91078162868875], [300.6012248986844, 451.76066889241594]], 'v': array([0.30388577, 0.09970683, 0.52461826])}),
 RectangleLoad({'x': [[196.90748880275964, 320.80144724613797], [304.3377719429589, 534.1659395352261]], 'v': array([0.3604886 , 0.59257619, 0.16958371])}),
 PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}),
 PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}),
 RectangleLoad({'x': [[54.030148129440626, 68.81200393827203], [97.24207408564268, 92.01555185605565]], 'v': array([0.42785045, 0.05710801, 0.07006977])}),
 RectangleLoad({'x': [[284.3334763991404, 23.082442148770088], [303.24173210567227, 55.03048055075901]], 'v': array([0.43239442, 0.8538399 , 0.15463183])}),
 PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 

In [9]:
[b.address for b in loads.cases()]

[['Group 1', 'Case 1'],
 ['Group 1', 'Case 2'],
 ['Group 1', 'Case 3'],
 ['Group 1', 'Case 4'],
 ['Group 1', 'Case 5'],
 ['Group 1', 'Case 6'],
 ['Group 1', 'Case 7'],
 ['Group 1', 'Case 8'],
 ['Group 1', 'Case 9'],
 ['Group 2', 'Case 10'],
 ['Group 2', 'Case 11'],
 ['Group 3', 'Case 12'],
 ['Group 3', 'Case 13'],
 ['Group 3', 'Case 14'],
 ['Group 4', 'Case 15'],
 ['Group 4', 'Case 16'],
 ['Group 4', 'Case 17'],
 ['Group 4', 'Case 18'],
 ['Group 4', 'Case 19'],
 ['Group 5', 'Case 20'],
 ['Group 5', 'Case 21'],
 ['Group 5', 'Case 22'],
 ['Group 5', 'Case 23'],
 ['Group 5', 'Case 24'],
 ['Group 5', 'Case 25'],
 ['Group 5', 'Case 26'],
 ['Group 5', 'Case 27'],
 ['Group 5', 'Case 28'],
 ['Group 6', 'Case 29'],
 ['Group 6', 'Case 30'],
 ['Group 6', 'Case 31'],
 ['Group 6', 'Case 32'],
 ['Group 7', 'Case 33'],
 ['Group 7', 'Case 34'],
 ['Group 7', 'Case 35'],
 ['Group 7', 'Case 36'],
 ['Group 7', 'Case 37'],
 ['Group 7', 'Case 38'],
 ['Group 7', 'Case 39'],
 ['Group 8', 'Case 40'],
 ['Group 

In [10]:
[b.parent for b in loads.cases()]

[LoadGroup({'Case 1': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 2': RectangleLoad({'x': [[104.58501059326417, 334.7006105962845], [165.25883822635677, 398.29663401029376]], 'v': array([0.53187527, 0.12620011, 0.48822345])}), 'Case 3': RectangleLoad({'x': [[48.959625850519174, 67.91078162868875], [300.6012248986844, 451.76066889241594]], 'v': array([0.30388577, 0.09970683, 0.52461826])}), 'Case 4': RectangleLoad({'x': [[196.90748880275964, 320.80144724613797], [304.3377719429589, 534.1659395352261]], 'v': array([0.3604886 , 0.59257619, 0.16958371])}), 'Case 5': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 6': PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}), 'Case 7': RectangleLoad({'x': [[54.030148129440626, 68.81200393827203], [97.24207408564268, 92.01555185605565]], 'v': array([0.42785045, 0.05710801, 0.07006977])}), 'Case 8': RectangleLoad({'x': [[284.3334763991404, 23.082442148770088], [303.24173210567227, 55.03048055075901]], 'v': array([0.4

In [11]:
len([id(b) for b in loads.blocks()]), len([id(b) for b in loads.blocks(inclusive=True)])

(1290, 1291)

In [12]:
[b.exclusive for b in loads.blocks()]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [13]:
[id(b) == id(loads) for b in loads.blocks()], [id(b) == id(loads) for b in loads.blocks(inclusive=True)]

([False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
 

In [14]:
[b.root is loads for b in loads.cases()]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [15]:
list(next(loads.values()).cases())

[PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}),
 RectangleLoad({'x': [[104.58501059326417, 334.7006105962845], [165.25883822635677, 398.29663401029376]], 'v': array([0.53187527, 0.12620011, 0.48822345])}),
 RectangleLoad({'x': [[48.959625850519174, 67.91078162868875], [300.6012248986844, 451.76066889241594]], 'v': array([0.30388577, 0.09970683, 0.52461826])}),
 RectangleLoad({'x': [[196.90748880275964, 320.80144724613797], [304.3377719429589, 534.1659395352261]], 'v': array([0.3604886 , 0.59257619, 0.16958371])}),
 PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}),
 PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 0, 0]}),
 RectangleLoad({'x': [[54.030148129440626, 68.81200393827203], [97.24207408564268, 92.01555185605565]], 'v': array([0.42785045, 0.05710801, 0.07006977])}),
 RectangleLoad({'x': [[284.3334763991404, 23.082442148770088], [303.24173210567227, 55.03048055075901]], 'v': array([0.43239442, 0.8538399 , 0.15463183])}),
 PointLoad({'x': [200.0, 400.0], 'v': [-100.0, 

In [138]:
import contextlib

load_groups = loads.values()

with contextlib.suppress(StopIteration):
    while True:
        results = plate.solve(next(load_groups), coords)
results

DeepDict({'Group 200': DeepDict({'Case 1176': <xarray.DataArray (index: 2601, component: 13)>
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.73378793e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.92718544e-05, ...,
         5.74318360e+00, -1.57405187e-03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  5.86274429e-05, ...,
         5.76459170e+00, -6.05283758e-03,  0.00000000e+00],
       ...,
       [-1.06881824e-18,  3.28250950e-20, -4.53411135e-05, ...,
         4.42947531e+00,  4.86755853e-03, -3.17926811e-18],
       [-5.39867370e-19,  3.32303273e-20, -2.26983303e-05, ...,
         4.44481307e+00,  2.91029636e-03,  5.65776592e-18],
       [-1.69081636e-33,  3.34535852e-20, -4.51823503e-20, ...,
         4.45062757e+00,  3.99692122e-17,  1.10979567e-17]])
Coordinates:
  * index      (index) int32 0 1 2 3 4 5 6 ... 2595 2596 2597 2598 2599 2600
  * component  (component) <U4 'UZ' 'ROTX' 'ROTY' 'CX' ... 

In [17]:
import dask

#del results

def task(group):
    return plate.solve(group, coords)

tasks = [dask.delayed(task)(group) for group in loads.values()]
results = dask.compute(*tasks)

results[-1]

DeepDict({'Group 200': DeepDict({'Case 1081': <xarray.DataArray (index: 2601, component: 13)>
array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.73378793e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.92718544e-05, ...,
         5.74318360e+00, -1.57405187e-03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  5.86274429e-05, ...,
         5.76459170e+00, -6.05283758e-03,  0.00000000e+00],
       ...,
       [-1.06881824e-18,  3.28250950e-20, -4.53411135e-05, ...,
         4.42947531e+00,  4.86755853e-03, -3.17926811e-18],
       [-5.39867370e-19,  3.32303273e-20, -2.26983303e-05, ...,
         4.44481307e+00,  2.91029636e-03,  5.65776592e-18],
       [-1.69081636e-33,  3.34535852e-20, -4.51823503e-20, ...,
         4.45062757e+00,  3.99692122e-17,  1.10979567e-17]])
Coordinates:
  * index      (index) int32 0 1 2 3 4 5 6 ... 2595 2596 2597 2598 2599 2600
  * component  (component) <U4 'UZ' 'ROTX' 'ROTY' 'CX' ... 